In [52]:
import pandas as pd 
import numpy as np
import os
#import cv2
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten, Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.metrics import Recall as recall
from keras import layers

import matplotlib.pyplot as plt
%matplotlib inline

### Preprocessing

In [28]:
train_path = "data/chest_xray/train"
test_path = "data/chest_xray/test"
val_path = "data/chest_xray/val"

In [29]:
# 
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [43]:
# 
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(200,200),
    batch_size = 32,
    color_mode = 'grayscale',
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_path, 
    target_size = (200, 200),
    batch_size = 32, 
    color_mode = 'grayscale',
    class_mode = 'categorical')

val_generator = test_datagen.flow_from_directory(
    val_path, 
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'grayscale',
    class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [9]:
#train_class_names = list(train_generator.class_indices.keys())
#train_class_names

['NORMAL', 'PNEUMONIA']

### Base Model

In [53]:
# Initializing the CNN
base_model = Sequential()

# 1st convolution layer and pooling
base_model.add(layers.Conv2D(32, (3, 3), input_shape=(200, 200, 1), activation = 'relu'))
base_model.add(MaxPooling2D(pool_size = (3, 3)))

# 2nd convolution layer and pooling
base_model.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
base_model.add(MaxPooling2D(pool_size = (3, 3)))

# plattening the layers
base_model.add(Flatten())

# adding the fully connected dense layer
base_model.add(Dense(64, activation = 'relu'))
base_model.add(Dropout(0.3))
# output layer
base_model.add(Dense(2, activation = 'sigmoid'))

# compiler
base_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=[recall()])

history = base_model.fit(train_generator, 
                        batch_size = 32,
                        epochs = 10,
                        validation_data = (val_generator))

# dropouts, keep it closer to the cnn , 1-2nd 

Epoch 1/10


2022-12-05 15:26:16.872314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 29s 177ms/step - loss: 0.3814 - recall_15: 0.8048 - val_loss: 0.5528 - val_recall_15: 0.7500
Epoch 2/10


2022-12-05 15:26:46.047272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 29s 176ms/step - loss: 0.2687 - recall_15: 0.8829 - val_loss: 0.4765 - val_recall_15: 0.9375
Epoch 3/10
163/163 [==============================] - 29s 176ms/step - loss: 0.2369 - recall_15: 0.9013 - val_loss: 0.7605 - val_recall_15: 0.6875
Epoch 4/10
163/163 [==============================] - 28s 174ms/step - loss: 0.2150 - recall_15: 0.9158 - val_loss: 0.6014 - val_recall_15: 0.6875
Epoch 5/10
163/163 [==============================] - 29s 176ms/step - loss: 0.2106 - recall_15: 0.9210 - val_loss: 1.1192 - val_recall_15: 0.5625
Epoch 6/10
163/163 [==============================] - 29s 176ms/step - loss: 0.1946 - recall_15: 0.9291 - val_loss: 1.0283 - val_recall_15: 0.6250
Epoch 7/10
163/163 [==============================] - 29s 175ms/step - loss: 0.2025 - recall_15: 0.9231 - val_loss: 0.4687 - val_recall_15: 0.8125
Epoch 8/10
163/163 [==============================] - 29s 175ms/step - loss: 0.1778 - recall_15: 0.9398 - val_loss: 0.6502 - val_

In [ ]:
# recall on the validation set it tells that the model is probably a bit overfitting, but as the validation set is too small
# I will try to play with the regularization parameters.



In [51]:
base_model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 66, 66, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 21, 21, 32)       0         
 g2D)                                                            
                                                                 
 flatten_16 (Flatten)        (None, 14112)             0         
                                                                 
 dense_32 (Dense)            (None, 64)              

In [ ]:
# 

In [ ]:
# define the stakeholders 